In [ ]:
import random, re, json, pathlib, logging, time, argparse, pprint 
import pandas as pd
import numpy as np
import circlify as crcf
import matplotlib.pyplot as plt
import importlib.resources as rsrc

#import ../postprocess.postprocess


In [ ]:
sdg_names = [f'SDG {count}' for count in range(1,18,1)]
sdg_color_ls = ['#e5243b','#dda63a','#4c9f38',
'#c5192d','#ff3a21','#26bde2',
'#fcc30b','#a21942','#fd6925',
'#dd1367','#fd9d24','#bf8b2e',
'#3F7E44','#0a97d9','#56c02b',
'#00689d','#19486a']
goal_colors = {label : color for label, color in zip(sdg_names,sdg_color_ls)}

target_list=['1.1', '1.2', '1.3', '1.4', '1.5', '1.a', '1.b',
 '2.1', '2.2', '2.3', '2.4', '2.5', '2.a', '2.b', '2.c',
 '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '3.a', '3.b', '3.c', '3.d',
 '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', '4.7', '4.a', '4.b', '4.c',
 '5.1', '5.2', '5.3', '5.4', '5.5', '5.6', '5.a', '5.b', '5.c',
 '6.1', '6.2', '6.3', '6.4', '6.5', '6.6', '6.a', '6.b',
 '7.1', '7.2', '7.3', '7.a', '7.b',
 '8.1', '8.2', '8.3', '8.4', '8.5', '8.6', '8.7', '8.8', '8.9', '8.10', '8.a', '8.b',
 '9.1', '9.2', '9.3', '9.4', '9.5', '9.a', '9.b', '9.c',
 '10.1', '10.2', '10.3', '10.4', '10.5', '10.6', '10.7', '10.a', '10.b', '10.c',
 '11.1', '11.2', '11.3', '11.4', '11.5', '11.6', '11.7', '11.a', '11.b', '11.c',
 '12.1', '12.2', '12.3', '12.4', '12.5', '12.6', '12.7', '12.8', '12.a', '12.b', '12.c',
 '13.1', '13.2', '13.3', '13.a', '13.b',
 '14.1', '14.2', '14.3', '14.4', '14.5', '14.6', '14.7', '14.a', '14.b', '14.c',
 '15.1', '15.2', '15.3', '15.4', '15.5', '15.6', '15.7', '15.8', '15.9', '15.a', '15.b', '15.c',
 '16.1', '16.2', '16.3', '16.4', '16.5', '16.6', '16.7', '16.8', '16.9', '16.10', '16.a', '16.b',
 '17.1', '17.2', '17.3', '17.4', '17.5', '17.6', '17.7', '17.8', '17.9', '17.10', '17.11', '17.12', '17.13', '17.14', '17.15', '17.16', '17.17', '17.18', '17.19']

target_colors = { target : goal_colors[f"SDG {target.split('.')[0]}"] for target in target_list}


In [ ]:
data = [
        {'id': 'Green Deal', 'datum': 6964195249, 'children' : [
              {'id' : "SDG 1", 'datum': 450448697,
                   'children' : [
                     {'id' : "Target 1.1", 'datum' : 308865000},
                     {'id' : "Target 1.2", 'datum' : 107550697},
                     {'id' : "Target 1.3", 'datum' : 34033000} 
                   ]},
              {'id' : "SDG 2", 'datum' : 278095425, 
                   'children' : [
                     {'id' : "Target 2.1", 'datum' : 192612000},
                     {'id' : "Target 2.2", 'datum' : 45349000},
                     {'id' : "Target 2.3", 'datum' : 40134425}
                   ]}],
        },
        {'id': 'Digital age', 'datum': 3564195249, 'children' : [
              {'id' : "SDG 3", 'datum' : 209246682,  
                   'children' : [
                     {'id' : "Target 3.1", 'datum' : 81757600},
                     {'id' : "Target 3.2", 'datum' : 65447374},
                     {'id' : "Target 3.3", 'datum' : 62041708}
                   ]},
              {'id' : "SDG 4", 'datum' : 311929000,  
                   'children' : [
                     {'id' : "Target 4.1", 'datum' : 154729000},
                     {'id' : "Target 4.2", 'datum' : 79221000},
                     {'id' : "Target 4.3", 'datum' : 77979000}
                   ]}],
        },
        {'id': 'Economy4People', 'datum': 4564195249, 'children' : [
              {'id' : "SDG 5", 'datum' : 2745929500,  
                   'children' : [
                     {'id' : "Target 5.1", 'datum' : 1336335000},
                     {'id' : "Target 5.2", 'datum' : 1178225000},
                     {'id' : "Target 5.3", 'datum' : 231369500}
                   ]},
              {'id' : "SDG 6", 'datum' : 3545929500,  
                   'children' : [
                     {'id' : "Target 6.1", 'datum' : 936335000},
                     {'id' : "Target 6.2", 'datum' : 1878225000},
                     {'id' : "Target 6.3", 'datum' : 271369500}
                   ]}]
        }
       ]

In [ ]:
# Compute circle positions thanks to the circlify() function
circles = crcf.circlify(
    data, 
    show_enclosure=False, 
    target_enclosure=crcf.Circle(x=0, y=0, r=1)
)

In [ ]:
# Create just a figure and only one subplot
fig, ax = plt.subplots(figsize=(14,14))

# Title
ax.set_title('Feh')

# Remove axes
ax.axis('off')

# Find axis boundaries
lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

rf={1:0.975, 2:0.95, 3:0.925} #reduction factor

# Print circles at pp level:
for circle in circles:
    if circle.level != 1:
      continue
    x, y, r = circle
    label = circle.ex["id"]
    ax.add_patch( plt.Circle((x, y), r*rf[circle.level], alpha=0.25, linewidth=2, color='purple'))

# Print circles at goal level:
for circle in circles:
    if circle.level != 2:
      continue
    x, y, r = circle
    label = circle.ex["id"]
    ax.add_patch( plt.Circle((x, y), r*rf[circle.level], alpha=0.5, linewidth=2, color=goal_colors[label]))

# Print circle at target level:
for circle in circles:
    if circle.level != 3:
      continue
    x, y, r = circle
    label = circle.ex["id"]
    color = re.sub(r'Target (\d{,2}).[a-z0-9]', r'SDG \1', label)
    label = label.replace('Target ','')
    ax.add_patch( plt.Circle((x, y), r*rf[circle.level], linewidth=0, color="white"))
    ax.add_patch( plt.Circle((x, y), r*rf[circle.level], alpha=0.5, linewidth=0, color=goal_colors[color]))
    plt.annotate(label, (x,y ), ha='center', color="white")

# Print labels for the continents
for circle in circles:
    if circle.level not in (1,2):
      continue
    x, y, r = circle
    label = circle.ex["id"]
    if circle.level == 2:
        plt.annotate(label, (x,y ) ,va='center', ha='center', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=.125))
    elif circle.level == 1:
        plt.annotate(label, (x,y+r), ha='center', color="black")



In [ ]:
p = pathlib.Path('..')
dirs=[x for x in p.iterdir() if x.is_dir()]
print(dirs)

In [ ]:
res_path=pathlib.Path('../output/PT_RRP_/output/6-results/results_.xlsx')
res_path.exists()
mock_target_df = pd.read_excel(res_path,sheet_name='target_dat') #pd.DataFrame({
#     'Name': target_list,
#     'Value': [np.random.randint(0, 100) for i in target_list]
# })
mock_target_df=mock_target_df[['Target','Count']]
mock_target_df.rename(columns={"Target": "Name", "Count": "Value"}, inplace=True)
mock_target_df =  mock_target_df[mock_target_df['Value'] != 0]

mock_goals = pd.read_excel(res_path,sheet_name='goal_overview')
mock_goals=mock_goals[['Goal','Count']]
mock_goals.rename(columns={"Goal": "Name", "Count": "Value"}, inplace=True)


In [ ]:
#build sunburst data

#Goal count from df
mock_goals_count = mock_goals['Value'].to_list()
mock_goals_labels = mock_goals['Name'].to_list()

#Target count from df
mock_tgts_count = mock_target_df['Value'].to_list()
mock_tgts_labels = mock_target_df['Name'].to_list()

values = mock_goals_count + mock_tgts_count
names = mock_goals_labels + mock_tgts_labels

#Assign parents
parents = []
for name in names:
    if 'SDG' in name:
        parents.append('')
    else:
        parent = f"SDG {name.split('.')[0]}"
        parents.append(parent)

data = dict(names=names, parents=parents, values=values)
for key in data.keys():
    print(key)

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

fig =px.sunburst(
    data,
    names='names',
    parents='parents',
    values='values',
    color='names',
    color_discrete_map={**goal_colors, **target_colors},
    branchvalues='remainder'
)
fig.update_traces(insidetextfont_color='black', selector=dict(type='sunburst'))
#fig.update_layout(uniformtext=dict(minsize=15, mode='show'))
fig.show()
fig.write_image('pt_rrp.png', width=450, height=450, scale=2)
# fig = go.Figure(go.Sunburst(
#     labels=data['names'],
#     parents=data['parents'],
#     values=data['values']),
#     color=data['names'],
#     color_discrete_map={**goal_colors, **target_colors}
# )
#


In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection="polar"))

size = 0.3
vals = np.array([[60., 32.], [37., 40.], [29., 10.]])
#normalize vals to 2 pi
valsnorm = vals/np.sum(vals)*2*np.pi
#obtain the ordinates of the bar edges
valsleft = np.cumsum(np.append(0, valsnorm.flatten()[:-1])).reshape(vals.shape)

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
inner_colors = cmap([1, 2, 5, 6, 9, 10])

outer = ax.bar(x=valsleft[:, 0],
       width=valsnorm.sum(axis=1), bottom=1-2*size, height=size,
       color=outer_colors, edgecolor='w', linewidth=1, align="edge")

# for bar, angle in zip(outer, valsleft[:, 0]):

#         #àbar.set_facecolor(color_dict[label])
#         bar.set_alpha(0.95)

#         # Labels are rotated. Rotation must be specified in degrees :(
#         rotation = np.rad2deg(angle)

#         # Flip some labels upside down
#         # alignment = ""
#         # if angle >= np.pi/2 and angle < 3*np.pi/2:
#         #     alignment = "right"
#         #     rotation = rotation + 180
#         # else: 
#         #     alignment = "left"

#         # Finally add the labels
#         label_string= f'feh {angle}' #f'{label} ({count})'
#         #y_pos= height - (labelPadding + len(label_string))

#         ax.text(
#             x=angle,
#             s=label_string,  
#             y= 1-2*size , #bar.get_height() + labelPadding,
#             ha='center', 
#             va='center', 
#             rotation=0, 
#             rotation_mode="anchor"#,
#             # fontsize=params_dict['fontsize'], 
#             # fontweight=params_dict['fontweight']
#             )

xs = [current] 

ax.text(
x=(valsleft[:, 0][2]+valsleft[:, 0][1])/2,#+(np.pi - valsleft[:, 0][2])/2,
s='Feh',  
y= 1-2*size , #bar.get_height() + labelPadding,
ha='center', 
va='center', 
rotation=0, 
rotation_mode="anchor"#,
# fontsize=params_dict['fontsize'], 
# fontweight=params_dict['fontweight']
)

#ax.bar_label(valsleft[:, 0], label_type='center')

inner = ax.bar(x=valsleft.flatten(),
       width=valsnorm.flatten(), bottom=1-size, height=size,
       color=inner_colors, edgecolor='w', linewidth=1, align="edge")

# for bar, angle in zip(inner, valsleft.flatten()):

#         #àbar.set_facecolor(color_dict[label])
#         bar.set_alpha(0.95)

#         # Labels are rotated. Rotation must be specified in degrees :(
#         rotation = np.rad2deg(angle)

#         # Flip some labels upside down
#         # alignment = ""
#         # if angle >= np.pi/2 and angle < 3*np.pi/2:
#         #     alignment = "right"
#         #     rotation = rotation + 180
#         # else: 
#         #     alignment = "left"

#         # Finally add the labels
#         label_string= 'feh' #f'{label} ({count})'
#         #y_pos= height - (labelPadding + len(label_string))

#         ax.text(
#             x=angle,
#             s=label_string,  
#             y= 1-size , #bar.get_height() + labelPadding,
#             ha='center', 
#             va='center', 
#             rotation=0, 
#             rotation_mode="anchor"#,
#             # fontsize=params_dict['fontsize'], 
#             # fontweight=params_dict['fontweight']
#             )

ax.set_axis_off()
plt.show()